# Select suitable MDL test: KVLCC2 **NO** speed

# Purpose
Find suitable roll decay tests from MDL, to reproduce in Motions later on.

# Methodology
Look at the results from the KLVCC2 ship.
* Load model tests from MDL
* Determine damping

# WIP - improvements
(WORK IN PROGRESS)
Use this section only if the notebook is not final.

Notable TODOs:
* todo 1
* todo 2
* todo 3

## Results
Describe and comment the most important results.

# Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables


In [ ]:
from mdldb.hdf5_file import Hdf5File
from rolldecayestimators.direct_estimator_cubic import EstimatorCubic,EstimatorQuadraticB
from sklearn.base import clone
from copy import deepcopy
import joblib

In [ ]:
db = database.get_db()

In [ ]:
sql = """
SELECT * from run
WHERE model_number='M5057-01-A' and test_type='roll decay' and project_number=40178362;
"""
df_rolldecays = pd.read_sql(sql=sql, con=db.engine, index_col='id')

df_rolldecays['ship_speed'].fillna(0, inplace=True)


In [ ]:
df_rolldecays

In [ ]:
df_rolldecays[['date','ship_speed']]

In [ ]:
df_rolldecays=df_rolldecays.loc[[21337,21338]].copy()

In [ ]:
fig,axes=plt.subplots(nrows=len(df_rolldecays))

for (id, row),ax in zip(df_rolldecays.iterrows(),axes):

    run = db.session.query(Run).get(int(id))
    run = database.load_run(run, save_as_example=False, prefer_hdf5=True)
    
    run.df.plot(y='phi', ax=ax)
    ax.set_ylabel(f"roll [{run.units['phi']}]")
    ax.grid(True)
    ax.set_ylim([-0.2,0.2])
    
plt.tight_layout()

In [ ]:
fig,axes=plt.subplots(ncols=len(df_rolldecays))

for (id, row),ax in zip(df_rolldecays.iterrows(),axes):

    run = db.session.query(Run).get(int(id))
    run = database.load_run(run, save_as_example=False, prefer_hdf5=True)
    
    run.track_plot(ax)
    
plt.tight_layout()

## Analyze the damping

In [ ]:
#run = db.session.query(Run).get(21338)
#run = db.session.query(Run).get(21339)
run = db.session.query(Run).get(21337)



#run = db.session.query(Run).get(21340)

	
run = database.load_run(run, save_as_example=False, prefer_hdf5=True)

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=1, minimum_score=0.999)
cutter = CutTransformer(phi_max=np.deg2rad(15), phi_min=np.deg2rad(2), phi1d_start_tolerance=0.015)
offset_transformer = OffsetTransformer()

In [ ]:
estimator_cubic = EstimatorCubic(fit_method='derivation')
estimator_quadratic_b = EstimatorQuadraticB(fit_method='derivation', maxfev=1000)

In [ ]:
steps = [
    ('filter',lowpass_filter),
    ('cutter', cutter), 
#    ('offset_transformer',offset_transformer),
    ('estimator', estimator_cubic)
]

pipline_cubic = Pipeline(steps=steps)

In [ ]:
steps = [
    ('filter',lowpass_filter),
    ('cutter', cutter), 
#    ('offset_transformer',offset_transformer),
    ('estimator', estimator_quadratic_b)
]

pipline_quadratic_b = Pipeline(steps=steps)

In [ ]:
models_cubic = OrderedDict()
scores = pd.DataFrame()

for (id, row),ax in zip(df_rolldecays.iterrows(),axes):

    run = db.session.query(Run).get(int(id))
    run = database.load_run(run, save_as_example=False, prefer_hdf5=True)
    
    pipline_cubic.fit(X=run.df[['phi']])
    scores.loc['models_cubic',id] = pipline_cubic.score(X=run.df[['phi']])
    
    models_cubic[id]=deepcopy(pipline_cubic)

In [ ]:
fig,ax=plt.subplots()

for id, model in models_cubic.items():
    
    estimator = model['estimator']
    estimator.plot_damping(ax=ax)
    
fig,ax=plt.subplots()
for id, model in models_cubic.items():
    
    estimator = model['estimator']
    estimator.plot_omega0(ax=ax, include_model_test=True)

## Quadratic

In [ ]:
models_quadratic = OrderedDict()

for (id, row),ax in zip(df_rolldecays.iterrows(),axes):

    run = db.session.query(Run).get(int(id))
    run = database.load_run(run, save_as_example=False, prefer_hdf5=True)
    
    pipline_quadratic_b.fit(X=run.df[['phi']])
    scores.loc['pipline_quadratic_b',id] = pipline_quadratic_b.score(X=run.df[['phi']])
    
    models_quadratic[id]=deepcopy(pipline_quadratic_b)

In [ ]:
fig,ax=plt.subplots()

colors = ['r','g','b']
markers = ['o','*','+']

for id, model in models_quadratic.items():
    
    estimator = model['estimator']
    estimator.plot_damping(ax=ax)
    
fig,ax=plt.subplots()
for id, model in models_quadratic.items():
    
    estimator = model['estimator']
    estimator.plot_omega0(ax=ax, include_model_test=True)

## Integration


### Quadratic

In [ ]:
estimator_quadratic_b_int = EstimatorQuadraticB(fit_method='integration', maxfev=100000, p0=estimator_quadratic_b.parameters)

steps = [
    ('filter',lowpass_filter),
    ('cutter', cutter), 
#    ('offset_transformer',offset_transformer),
    ('estimator', estimator_quadratic_b_int)
]

pipline_quadratic_b_int = Pipeline(steps=steps)

In [ ]:
models_quadratic_b_int = OrderedDict()

for (id, row),ax in zip(df_rolldecays.iterrows(),axes):

    run = db.session.query(Run).get(int(id))
    run = database.load_run(run, save_as_example=False, prefer_hdf5=True)
    
    pipline_quadratic_b_int.fit(X=run.df[['phi']])
    scores.loc['pipline_quadratic_b_int',id] = pipline_quadratic_b_int.score(X=run.df[['phi']])
    
    models_quadratic_b_int[id]=deepcopy(pipline_quadratic_b_int)

In [ ]:
fig,ax=plt.subplots()

colors = ['r','g','b']
markers = ['o','*','+']

for id, model in models_quadratic_b_int.items():
    
    estimator = model['estimator']
    estimator.plot_damping(ax=ax)
    
fig,ax=plt.subplots()
for id, model in models_quadratic.items():
    
    estimator = model['estimator']
    estimator.plot_omega0(ax=ax, include_model_test=True)

### Cubic

In [ ]:
estimator_cubic_int = EstimatorCubic(fit_method='integration', maxfev=100000, p0=estimator_cubic.parameters)

steps = [
    ('filter',lowpass_filter),
    ('cutter', cutter), 
#    ('offset_transformer',offset_transformer),
    ('estimator', estimator_cubic_int)
]

pipline_cubic_int = Pipeline(steps=steps)

In [ ]:
models_cubic_int = OrderedDict()

for (id, row),ax in zip(df_rolldecays.iterrows(),axes):

    run = db.session.query(Run).get(int(id))
    run = database.load_run(run, save_as_example=False, prefer_hdf5=True)
    
    pipline_cubic_int.fit(X=run.df[['phi']])
    scores.loc['models_cubic_int',id] = pipline_cubic_int.score(X=run.df[['phi']])
    
    models_cubic_int[id]=deepcopy(pipline_cubic_int)

In [ ]:
fig,ax=plt.subplots()

colors = ['r','g','b']
markers = ['o','*','+']

for id, model in models_cubic_int.items():
    
    estimator = model['estimator']
    estimator.plot_damping(ax=ax)
    
fig,ax=plt.subplots()
for id, model in models_cubic_int.items():
    
    estimator = model['estimator']
    estimator.plot_omega0(ax=ax, include_model_test=True)

In [ ]:
scores

In [ ]:
model = models_cubic_int[21338]

In [ ]:
scale_factor = run.model.scale_factor

meta_data = {
    'Volume':run.loading_condition.Volume/(scale_factor**3),
    'GM':run.loading_condition.gm/scale_factor,
}

estimator = model['estimator']
result_for_database = estimator.result_for_database(meta_data=meta_data)
result_for_database

In [ ]:
joblib.dump(model, '../models/KVLCC2_0_speed.pkl')

In [ ]:
result_for_database['omega0']

In [ ]:
result_for_database['omega0']/np.sqrt(run.model.scale_factor)

In [ ]:
phi_a = np.deg2rad(10)

B_e = lambdas.B_e_lambda_cubic(B_1=result_for_database['B_1'],
                         B_2=result_for_database['B_2'],
                         B_3=result_for_database['B_3'],
                         omega0=result_for_database['omega0'],
                         phi_a = phi_a)

In [ ]:
g=9.81
rho=1000
Volume = run.loading_condition.Volume/(scale_factor**3)
beam = run.ship.beam/scale_factor
lambdas.B_hat_lambda(B=B_e, Disp=Volume, beam=beam,g=g, rho=rho)

## $A_{44}$

In [ ]:
result_for_database['A_44']

In [ ]:
mass = run.loading_condition.Volume*rho
mass*run.loading_condition.KXX**2

In [ ]:
np.sqrt(result_for_database['A_44']/mass)

In [ ]:
result_for_database['A_44']/(mass*run.loading_condition.KXX**2)

In [ ]:
run.loading_condition.Volume/(run.ship.lpp*run.ship.beam*run.loading_condition.TA)

In [ ]:
run.ship.lpp

In [ ]:
run.ship.beam

In [ ]:
run.loading_condition.TA

In [ ]:
equations.A44

In [ ]:
omega0 = result_for_database['omega0']
scale_factor = run.model.scale_factor
GM = run.loading_condition.gm/scale_factor
m = run.loading_condition.Volume/(scale_factor**3)*rho

A_44 = lambdas.A44_lambda(GM=GM, g=g, m=m, omega0=omega0)
A_44

In [ ]:
kxx = run.loading_condition.KXX/scale_factor #???
I_44 = m*kxx**2
I_44

In [ ]:
A_44/I_44